In [3]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model
from tqdm import tqdm

In [4]:
# Preprocess the dataset
# Load the finalized dataset
df = pd.read_csv("../Datasets/pokedex_final.csv")

# Combine type1 and type2 into a list
df['type_list'] = df[['type1', 'type2']].values.tolist()
df['type_list'] = df['type_list'].apply(lambda x: [t for t in x if pd.notna(t) and t != ''])

# Encode the types with MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['type_list'])

# Tokenize enhanced_info using BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenization (batch)
encodings = tokenizer(
    list(df['enhanced_info']),
    padding=True,
    truncation=True,
    max_length=256,
    return_tensors='tf'
)

# Normalize numeric features
stats_cols = ['hp', 'attack', 'defense', 's_attack', 'speed', 'height', 'weight']
scaler = StandardScaler()
X_stats = scaler.fit_transform(df[stats_cols])


In [ ]:
# Split into train and test sets
X_train_text, X_test_text = {}, {}
train_idx, test_idx = train_test_split(np.arange(len(df)), test_size=0.2, random_state=42)

X_train_text['input_ids'] = tf.gather(encodings['input_ids'], train_idx)
X_train_text['attention_mask'] = tf.gather(encodings['attention_mask'], train_idx)
X_test_text['input_ids'] = tf.gather(encodings['input_ids'], test_idx)
X_test_text['attention_mask'] = tf.gather(encodings['attention_mask'], test_idx)

X_train_stats = tf.gather(X_stats, train_idx)
X_test_stats = tf.gather(X_stats, test_idx)

y_train = tf.gather(y, train_idx)
y_test = tf.gather(y, test_idx)

# Build the BERT + Stats model
bert_model = TFBertModel.from_pretrained("bert-base-uncased")

input_ids = Input(shape=(256,), dtype=tf.int32, name='input_ids')
attention_mask = Input(shape=(256,), dtype=tf.int32, name='attention_mask')
stats_input = Input(shape=(X_stats.shape[1],), dtype=tf.float32, name='stats_input')

bert_output = bert_model(input_ids, attention_mask=attention_mask)[1]  # pooled output
combined = Concatenate()([bert_output, stats_input])

hidden = Dense(128, activation='relu')(combined)
output = Dense(len(mlb.classes_), activation='sigmoid')(hidden)

model = Model(inputs=[input_ids, attention_mask, stats_input], outputs=output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

# Train the model
model.fit(
    [X_train_text['input_ids'], X_train_text['attention_mask'], X_train_stats],
    y_train,
    validation_data=([X_test_text['input_ids'], X_test_text['attention_mask'], X_test_stats], y_test),
    epochs=5,
    batch_size=16
)

# Save model and label binarizer
model.save("/mnt/data/bert_stats_pokemon_model")
pd.DataFrame(mlb.classes_, columns=['Type']).to_csv("/mnt/data/type_classes.csv", index=False)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 256)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 256)]                0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPooli   1094822   ['input_ids[0][0]',           
 )                           ngAndCrossAttentions(last_   40         'attention_mask[0][0]']      
                             hidden_state=(None, 256, 7                                       